In [30]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import pandas as pd
import re

options = webdriver.ChromeOptions()
driver = webdriver.Chrome(options=options)

url = "https://results.chicagomarathon.com/2024/?page=1&event=MAR&event_main_group=runner&num_results=1000&pid=search&search%5Bsex%5D=%25&search%5Bage_class%5D=%25&search%5Bnation%5D=%25&search_sort=place_nosex"
driver.get(url)
time.sleep(4)

resultados = []

# Procesar solo los primeros 10 atletas
atletas = driver.find_elements(By.CSS_SELECTOR, "li.list-group-item.row")[:10]

for atleta in atletas:
    try:
        place_overall = atleta.find_element(By.CSS_SELECTOR, ".list-field.type-place.place-secondary").text
    except:
        place_overall = "Sin posición general"

    try:
        place_gender = atleta.find_element(By.CSS_SELECTOR, ".list-field.type-place.place-primary").text
    except:
        place_gender = "Sin posición género"

    try:
        nombre_pais = atleta.find_element(By.CSS_SELECTOR, ".list-field.type-fullname a").text
        nombre = re.sub(r"\s*\([A-Z]{3}\)", "", nombre_pais).strip()
        pais_match = re.search(r"\(([A-Z]{3})\)", nombre_pais)
        pais = pais_match.group(1) if pais_match else "Sin país"
    except:
        nombre = "Sin nombre"
        pais = "Sin país"

    try:
        division_block = atleta.find_element(By.CSS_SELECTOR, ".list-field.type-age_class").text
        division = division_block.replace("Division", "").strip()
    except:
        division = "Sin división"

    # ✅ Nuevo método para capturar el tiempo Finish
    try:
        finish_block = atleta.find_element(By.XPATH, ".//div[contains(@class, 'type-time') and .//div[text()='Finish']]")
        html = finish_block.get_attribute("innerHTML")
        match = re.search(r'</div>\s*([0-9:]+)', html)
        finish = match.group(1).strip() if match else "Sin tiempo"
    except:
        finish = "Sin tiempo"

    resultados.append({
        "place_overall": place_overall,
        "place_gender": place_gender,
        "nombre": nombre,
        "pais": pais,
        "division": division,
        "finish": finish
    })

driver.quit()

df = pd.DataFrame(resultados)

In [32]:
df.to_csv("../data/ejemplo_10_resultados_chicago_marathon_2024.csv", index=False)

In [33]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import pandas as pd
import re

def scrape_chicago_test(genero, url):
    options = webdriver.ChromeOptions()
    driver = webdriver.Chrome(options=options)

    print(f"\n🔍 Scrapeando 10 atletas – Género: {genero}")
    driver.get(url)
    time.sleep(4)

    resultados = []
    atletas = driver.find_elements(By.CSS_SELECTOR, "li.list-group-item.row")[:10]

    for atleta in atletas:
        try:
            place_overall = atleta.find_element(By.CSS_SELECTOR, ".list-field.type-place.place-secondary").text
        except:
            place_overall = "Sin posición general"

        try:
            place_gender = atleta.find_element(By.CSS_SELECTOR, ".list-field.type-place.place-primary").text
        except:
            place_gender = "Sin posición género"

        try:
            nombre_pais = atleta.find_element(By.CSS_SELECTOR, ".list-field.type-fullname a").text
            nombre = re.sub(r"\s*\([A-Z]{3}\)", "", nombre_pais).strip()
            pais_match = re.search(r"\(([A-Z]{3})\)", nombre_pais)
            pais = pais_match.group(1) if pais_match else "Sin país"
        except:
            nombre = "Sin nombre"
            pais = "Sin país"

        try:
            division_block = atleta.find_element(By.CSS_SELECTOR, ".list-field.type-age_class").text
            division = division_block.replace("Division", "").strip()
        except:
            division = "Sin división"

        try:
            finish_block = atleta.find_element(By.XPATH, ".//div[contains(@class, 'type-time') and .//div[text()='Finish']]")
            html = finish_block.get_attribute("innerHTML")
            match = re.search(r'</div>\s*([0-9:]+)', html)
            finish = match.group(1).strip() if match else "Sin tiempo"
        except:
            finish = "Sin tiempo"

        resultados.append({
            "place_overall": place_overall,
            "place_gender": place_gender,
            "nombre": nombre,
            "pais": pais,
            "division": division,
            "finish": finish,
            "gender": genero
        })

    driver.quit()
    return pd.DataFrame(resultados)

# URLs
url_hombres = "https://results.chicagomarathon.com/2024/?page=1&event=MAR&event_main_group=runner&num_results=1000&pid=list&search%5Bsex%5D=M&search%5Bage_class%5D=%25"
url_mujeres = "https://results.chicagomarathon.com/2024/?page=1&event=MAR&event_main_group=runner&num_results=1000&pid=list&search%5Bsex%5D=W&search%5Bage_class%5D=%25"

# Ejecutar prueba
df_hombres = scrape_chicago_test("M", url_hombres)
df_mujeres = scrape_chicago_test("W", url_mujeres)

# Concatenar resultados
df_chicago = pd.concat([df_hombres, df_mujeres], ignore_index=True)

print("\n✅ Prueba completada. Archivo: 'prueba_chicago_10_M_10_W.csv'")


🔍 Scrapeando 10 atletas – Género: M

🔍 Scrapeando 10 atletas – Género: W

✅ Prueba completada. Archivo: 'prueba_chicago_10_M_10_W.csv'


In [34]:
df_chicago

,place_overall,place_gender,nombre,pais,division,finish,gender
0,Place Overall,Place Gender,Sin nombre,Sin país,,Sin tiempo,M
1,1,1,"Korir, John",KEN,25-29,02:02:44,M
2,2,2,"Esa, Huseydin Mohamed",ETH,20-24,02:04:39,M
3,3,3,"Kipruto, Amos",KEN,30-34,02:04:50,M
4,4,4,"Ngetich, Vincent",KEN,25-29,02:05:16,M
5,5,5,"Ebenyo, Daniel",KEN,25-29,02:06:04,M
6,6,6,"Hosoya, Kyohei",JPN,25-29,02:07:20,M
7,7,7,"Albertson, Cj",USA,30-34,02:08:17,M
8,8,8,"Sadakata, Toshiki",JPN,30-34,02:08:22,M
9,9,9,"Masai, Alex",KEN,25-29,02:08:51,M


In [35]:
df_chicago.to_csv("../data/ejemplo_10_resultados_chicago_marathon_2024.csv", index=False)